In [49]:
import numpy as np
import itertools

In [2]:
def heaviside(x):
    return np.array(1 * (x >= 0))

In [98]:
def extension_bool(p,V):
    """
    Compute a boolean vector that represents the extension of p in V
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        extension - a vector of shape (L) with elements from {1,0}.  extension[l]=1 iff V[l,:] is in the extension of p
    """
    
    E = np.dot(V,p)/np.sum(abs(p))
    return heaviside(E-1)
def extension_mat(p,V):
    """
    Returns a matrix representing the extension of p in V
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        A - A matrix of shape (N,M) with elements from {-1,1}, where N <=L, and the rows of A are the rows of V in the
            extension of p
    """
    
    bool_vec = extension_bool(p,V)
    return V[bool_vec>0,:]
def check_equivalent(p,q,V):
    """
    Checks if p and q have the same extension in V
    
    Inputs:
        p,q - vectors of shape (M,) with elements from {-1,0,1}.  The partially specified feature vectors
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        match - bool, whether or not p and q have the same extension 
    """
    p_bool = extension_bool(p,V)
    q_bool = extension_bool(q,V)
    M = np.shape(q_bool)[0]
    return all([p_bool[i]==q_bool[i] for i in range(M)])

def mutate_p(p,abstraction_set,specification_set):
    """
    Returns q that is equal to p with mutations (specifications and generalizations applied)
    
    Inputs:
        p - the partially specified feature vector to be mutated
        abstraction_set - a set of indices to flip currently specified values of p to zero
        specification_set - a set of tuples (index,value) to change currently unspecified values of p to the given value
    Outputs:
        q - the mutated vector
    """
    
    q = np.copy(p)
    for abstraction in abstraction_set:
        q[abstraction]=0
    for specification_tuple in specification_set:
        q[specification_tuple[0]]=specification_tuple[1]
    return q
        
def equivalence_class(p,V):
    """
    Given a set of feature vectors V and a query partially specified feature vector p, return all partially
    specified vectors q that have the same extension in V as p.
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        R - a list of vectors where R[i] has shape (M,) and all vectors in R have the same extension in V as p
    """
    
    M = np.shape(p)[0]
    L = np.shape(V)[0]
    V_sums = np.sum(V,axis=0)
    bool_p = extension_bool(p,V)
    
    A = V[bool_p>0,:]#shape is (N,M)
    N = np.shape(A)[0]
    A_sums = np.sum(A,axis=0)
    
    allowed_specifications = []
    prespecified_indices = []
    for i in range(M):
        if p[i]==0:
            #we can flip this position to a signed value if only if all A[:,i] or equivalently A_sums[i]== +/- N
            if A_sums[i]==N:
                allowed_specifications.append((i,1)) #they're all 1 so we can mutate p[i] to 1
            elif A_sums[i]==-N:
                allowed_specifications.append((i,-1)) #they're all -1 so we can mutate p[i] to -1
        else:
            prespecified_indices.append(i)
            
    allowed_specification_sets = []
    for r in range(len(allowed_specifications)):
        combinations = itertools.combinations(allowed_specifications,r+1)
        for subset in combinations:
            allowed_specification_sets.append(list(subset))
    allowed_specification_sets.append([])
    
    #which indices of p can be zeroed out without changing the extension of p in V?
    
    T = L - N
    B = V[bool_p<=0,:]#shape is (T,M), p is not compatible with any vector in B
    
    #now we find all subsets q of the specified indices such that if we zeroed out every index of p contained in q, the
    #new vector would be consistent with an element of B
    #this accounts to looking at each element b (row) of B, and finding all the indices where p disagrees 
    #with that element and adding that set of indices to an accumulating set
    forbidden_index_subsets_ = []
    
    for t_val in range(T):
        clash_points = []
        for m_val in range(M):
            if B[t_val,m_val]*p[m_val] == -1:
                clash_points.append(m_val)
        forbidden_index_subsets_.append(frozenset(clash_points))
    forbidden_index_subsets = set(forbidden_index_subsets_)
    
    
    my_queue = [set([i]) for i in prespecified_indices]
    prespecified_index_singletons = [frozenset([i]) for i in prespecified_indices]
    legal_index_sets=[] #will eventually hold all the allowed abstraction sets
    while len(my_queue)>0:
        trial_set = my_queue.pop(0)
        frozen_trial_set = frozenset(trial_set)
        if not (frozen_trial_set in forbidden_index_subsets):
            #it's a legal set of indices to all zero out
            legal_index_sets.append(frozen_trial_set)
            #we also add this set combined with each possible singleton back into the queue for later testing
            for singleton in prespecified_index_singletons:
                if (not singleton.issubset(trial_set)) and (not singleton in forbidden_index_subsets):
                    my_queue.append(trial_set.union(singleton))
    legal_index_sets.append(frozenset())
    
    #Now I must combine the set of allowable specification sets and the set of allowable generalization sets
    #to creat all possible mutated vectors
    
    equivalence_class = []
    for specification_set in allowed_specification_sets:
        for abstraction_set in legal_index_sets:
            q = mutate_p(p,abstraction_set,specification_set)
            equivalence_class.append(q)
    return equivalence_class
    
    #return allowed_specification_sets,legal_index_sets
    
    """
    #now we must check which subsets of the specified indices can be zeroed out without making the resulting 
    # 'de-specified' vector consistent any vectors in B
    
    #first we create all nonempy subsets of the already specified indices of p and turn it into a set of frozen sets
    specified_index_subsets_ = []
    for r in range(len(test)):
        combinations = itertools.combinations(prespecified_indices,r+1)
        for subset in combinations:
            specified_index_subsets_.append(frozenset(subset))
    specified_index_subsets = set(specified_index_subsets_)
    
    #now we find all subsets q of the specified indices such that if we zeroed out every index of p contained in q, the
    #new vector would be consistent with an element of B
    #this accounts to looking at each element b (row) of B, and finding all the indices where p disagrees 
    #with that element and adding that set of indices to an accumulating set
    
    forbidden_index_subsets_ = []
    for t_val in range(T):
        clash_points = []
        for m_val in range(M):
            if B[t_val,m_val]*p[m_val] == -1:
                clash_points.append(m_val)
        forbidden_index_subsets_.append(frozenset(clash_points))
    forbidden_index_subsets = set(forbidden_index_subsets_)
    
    #we now take the set difference
    
    legal_index_sets = specified_index_subsets.difference(forbidden_index_subsets)
    
    return allowed_specifications, specified_index_subsets, forbidden_index_subsets, legal_index_sets
    """
    
    

In [100]:
p = np.array([-1,0,1])
V = np.array([
    [-1,1,1],
    [1,1,1],
    [-1,1,1]
])
extension_mat(p,V)

array([[-1,  1,  1],
       [-1,  1,  1]])

In [101]:
equivalence_class(p,V)

[array([-1,  1,  0]),
 array([-1,  1,  1]),
 array([-1,  0,  0]),
 array([-1,  0,  1])]

In [13]:
test = np.array([1,0,1,0,1])
B[test>0,:]

array([[0, 1],
       [4, 5],
       [8, 9]])

In [97]:
p = [1,0,1,0,-1]
specification_set = []
abstraction_set = {}
mutate_p(p,abstraction_set,specification_set)

array([ 1,  0,  1,  0, -1])

In [37]:
test = [1,2,3]
specified_index_sets = []
for r in range(len(test)):
    combinations = itertools.combinations(test,r+1)
    for subset in combinations:
        specified_index_sets.append(frozenset(subset))
specified_index_sets = set(specified_index_sets)

In [38]:
specified_index_sets

{frozenset({3}),
 frozenset({1, 2}),
 frozenset({2, 3}),
 frozenset({1}),
 frozenset({1, 3}),
 frozenset({2}),
 frozenset({1, 2, 3})}

In [53]:
test = set(
    [frozenset({1,2}),frozenset({1}),frozenset({1,2,3})]
    )

In [56]:
frozenset([1,2,3]) in test

True

In [65]:
b = set({1,2})
a = b.union({3})
print a

set([1, 2, 3])


In [74]:
b.issubset(a)

True